# Import `pygismo` and other packages

We import `pygismo`

In [ ]:
import pygismo as gs

We also import `numpy` and `matplotlib`

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

# Constructing a 2D basis

We use the same B-spline basis as in the previous example, with knot vectors:

$\Xi_u = \Xi_v = \{0,0,0,0.25,0.50,0.75,1,1,1\}$


In [ ]:
kv_u = kv_v = gs.nurbs.gsKnotVector(np.array([0,0,0,0.25,0.50,0.75,1,1,1]),2)

In [ ]:
tbasis = gs.nurbs.gsTensorBSplineBasis2(kv_u,kv_v)


# Definining a surface

Similar to defining a curve, we define a matrix of control points to construct a surface. For a basis of size $N$, the coefficient matrix is defined as an $N\times d$ matrix, with $d$ the geometric dimension. Furthermore, we initialize the coefficient matrix with $N\times d$ zeros.

Since our basis is composed of the same knot vector in both directions, the number of functions in each direction is $n=\sqrt{N}$.

In [ ]:
coefs = np.zeros((tbasis.size(),3))
n = np.sqrt(tbasis.size())

Then, we define the control points in a uniformly spaced grid. We use `numpy.linspace` and `numpy.meshgrid` for this

In [ ]:
x = np.linspace(0,1,int(n))
X,Y = np.meshgrid(x,x)
coefs[:,0] = X.flatten()
coefs[:,1] = Y.flatten()

Since the coefficient matrix is initialized with zeros, we modify a few coefficients to displace the points out-of-plane.

In [ ]:
coefs[12,2] = -1
coefs[16,2] = 1
coefs[22,2] = -1
coefs[26,2] = 10

Having defined the coefficient matrix, we can make a tensor B-splne surface using the basis and the coefficient matrix.

In [ ]:
surf = gs.nurbs.gsTensorBSpline2(tbasis,coefs)

# Plotting the 3D surface

We can use `matplotlib` to plot the surface here in python. As always, we start by creating a set of parametric points. Here, they stored in `x` and `y`, on their turn used to construct a `numpy.meshgrid`. The grid is used to create a matrix of parametric points to evaluate the surface on. The matrix stores the points as columns.

In [ ]:
N = M = 100
x = np.linspace(0,1,N)
y = np.linspace(0,1,M)
XX, YY = np.meshgrid(x,y,indexing='xy')
pts = np.stack((XX.flatten(),YY.flatten()))

print(pts)

The surface is simply evaluated using `eval`, and the result is properly reshaped into the format of the `numpy.meshgrid`:

In [ ]:
S = surf.eval(pts)
ZZ = S[2,:].reshape((N,M))

Then, we use `matplotlib` to plot the surface and its control points

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection ='3d')
ax.plot_surface(XX,YY,ZZ,cmap=cm.coolwarm)
ax.scatter(surf.coefs()[:,0],surf.coefs()[:,1],surf.coefs()[:,2])
plt.show()

# Reading a surface from a file

As for the curve example, we can read the surface from XML.

In [ ]:
filename = "filedata/surface_example.xml"
filedata = gs.io.gsFileData(filename)

The surface is stored with id 0 in the file. Hence, to read it into an object, we do

In [ ]:
XMLsurf = gs.nurbs.gsTensorBSpline2()
filedata.getId(0,XMLsurf)

print(XMLsurf.basis().size())
print(XMLsurf.coefs())

Using the same plotting commands as before, we can plot the result

In [ ]:
N = M = 100
x = np.linspace(0,1,N)
y = np.linspace(0,1,M)
XX, YY = np.meshgrid(x,y,indexing='xy')
pts = np.stack((XX.flatten(),YY.flatten()))

S = XMLsurf.eval(pts)
ZZ = S[2,:].reshape((N,M))

fig = plt.figure()
ax = fig.add_subplot(projection ='3d')
ax.plot_surface(XX,YY,ZZ,cmap=cm.coolwarm)
ax.scatter(XMLsurf.coefs()[:,0],XMLsurf.coefs()[:,1],XMLsurf.coefs()[:,2])
plt.show()

# Plotting the result in Paraview

Alternatively, we can use Paraview to plot our surface. We simply use `pygismo`'s buit-in functionality

In [ ]:
gs.io.gsWriteParaview(surf,"output/surf")

This creates the files `surf0.vts` and `surf.pvd`. When we open `surf.pvd` in paraview, we see the surface.